In [26]:
#imports ddependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [27]:
#initiate url for zolo site for only houses in toronto
url = 'https://www.zolo.ca/toronto-real-estate/houses'
# get all href links for indiviual houses and store in list
# go thru all pages one by one 
results = []
response = requests.get(url)
for x in range(54):
    
    soup = bs(response.text, 'lxml')
    
    list_items = soup.find_all("a", class_='address link-primary')

    # get all href links for indiviual houses and store in list in current page
  
    for item in list_items:
        link = item['href'] 
        results.append(link)
    #construct link for next page
    next_url = 'https://www.zolo.ca/toronto-real-estate/houses/page-' + str(x +1)
    request = requests.get(next_url)
    if request.status_code == 200:
        response = requests.get(next_url)
        #browser.visit(next_url)
    else:
        print("url collection compleated")
        break
    #print(next_url)

In [28]:
#check point for links
for link in results:
    print(link)

https://www.zolo.ca/toronto-real-estate/28-caswell-drive
https://www.zolo.ca/toronto-real-estate/18-vanbrugh-avenue
https://www.zolo.ca/toronto-real-estate/53-northbrook-road
https://www.zolo.ca/toronto-real-estate/3-beaconhill-road-south
https://www.zolo.ca/toronto-real-estate/30-hedgewood-road
https://www.zolo.ca/toronto-real-estate/1023-kipling-avenue
https://www.zolo.ca/toronto-real-estate/82-whistling-hills-drive
https://www.zolo.ca/toronto-real-estate/18-hill-crescent
https://www.zolo.ca/toronto-real-estate/118-citadel-drive
https://www.zolo.ca/toronto-real-estate/92-midland-avenue
https://www.zolo.ca/toronto-real-estate/18-esmond-crescent
https://www.zolo.ca/toronto-real-estate/283-kenilworth-avenue
https://www.zolo.ca/toronto-real-estate/273-manor-road-east
https://www.zolo.ca/toronto-real-estate/118-ava-road
https://www.zolo.ca/toronto-real-estate/44-ash-crescent
https://www.zolo.ca/toronto-real-estate/61-monterrey-drive
https://www.zolo.ca/toronto-real-estate/16-woodlee-road


In [29]:
#get below house info into the list/dictinary for page
######'address', 'city', 'hoot', 'Type', 'Style, 'Size', 'Lot Size',
######'Age', 'Taxes', 'Walk Score', 'MLS®#', 'Days on Site', 'Listed By'
#finally load in the dataframe

#initialized empty df outsite of loop
house_df = pd.DataFrame()

#ittrated all links in for loop and collect datas abouve
for link in results:
    
    response = requests.get(link)
    soup = bs(response.text, 'lxml')
    
    # Retrieve address
    address= soup.find('h1', class_="address xs-text-2 sm-text-1 truncate bold").text.rstrip().lstrip()
    # Retrieve city and neighborhood 
    area = soup.find('div', 'a', class_= "area xs-text-5 sm-text-4 truncate").text.rstrip().lstrip().split(", ")
    try:        
        city = area[0]
        hoot = area[1]
    except:
        city = 'none'
        hoot = 'none'
    
    # Retrieve price    #div class="xs-text-2 sm-text-1 bold xs-inline sm-block xs-mr05 sm-mr0
    price = soup.find('div', class_="xs-text-2 sm-text-1 bold xs-inline sm-block xs-mr05 sm-mr0").text.rstrip().lstrip()
    
    # Retrieve rest of the info from dl dd dt tags 
    #collect info into the dictinary 
    house_info = soup.find_all('div',  {'class':"column-container sm-column-count-2 column-gap"})
    
    house_dict={"address": address,"city" :city, "hoot": hoot, 'price':price}
    
    house_info_keys = []
    for i in house_info[0].find_all('dt'):
        house_info_keys.append(i.text)
        
    house_info_values = []
    for i in house_info[0].find_all('dd'):
        house_info_values.append(i.text.rstrip().lstrip())
    
    
    for key in house_info_keys:
        for value in house_info_values:
            house_dict[key] = value
            house_info_values.remove(value)
            break 
            
    house_df = house_df.append(house_dict, ignore_index=True)


In [30]:
#validate data
house_df.head()

,Age,Days on Site,Listed By,Lot Size,MLS®#,Size,Style,Taxes,Type,Walk Score,address,city,hoot,price,Business Type,Property Type,Zoning,Strata Fees,Year Built
0,0-5 years,"5 (Dec 11, 2019)","Zolo Realty, Brokerage",70.00 x 115 Feet,C4651523,5000 sqft,3-Storey,"$10,920 /yr",Detached,20,28 Caswell Drive,Toronto,Willowdale East,"$3,888,888",NaN,NaN,NaN,NaN,NaN
1,No Data,"20 (Nov 26, 2019)","Zolo Realty, Brokerage",33.00 x 120 Feet,E4642347,No Data,2-Storey,"$3,115 /yr",Detached,77,18 Vanbrugh Avenue,Toronto,Birchcliffe-cliffside,"$799,000",NaN,NaN,NaN,NaN,NaN
2,0-5 years,"27 (Nov 19, 2019)","Zolo Realty, Brokerage",32.00 x 107 Feet,E4638092,1500-2000 sqft,2-Storey,"$4,038 /yr",Detached,62,53 Northbrook Road,Toronto,East York,"$1,699,000",NaN,NaN,NaN,NaN,NaN
3,51-99 years,"1 (Dec 16, 2019)","Royal Star Realty Inc., Brokerage",50.00 x 110 Feet,W4654296,2000-2500 sqft,Backsplit 4,"$3,246 /yr",Detached,53,3 Beaconhill Road,Toronto,Mount Olive-silverstone-jamestown,"$995,400",NaN,NaN,NaN,NaN,NaN
4,No Data,"1 (Dec 16, 2019)","Jdl Realty Inc., Brokerage",56.50 x 145 Feet,C4654304,No Data,2-Storey,"$10,077 /yr",Detached,49,30 Hedgewood Road,Toronto,Bridle Path-sunnybrook-york Mills,"$2,889,000",NaN,NaN,NaN,NaN,NaN


In [22]:
#testing for not scapping everytine
#dataframe to out put to csv
house_df.to_csv("output_1.csv")

In [25]:
#csv to dataframe to out put to csv
house_df_2 =pd.DataFrame.from_csv("output_1.csv")
house_df_2.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


,Age,Days on Site,Listed By,Lot Size,MLS®#,Size,Style,Taxes,Type,Walk Score,address,city,hoot,price,Business Type,Property Type,Zoning,Strata Fees,Year Built
0,0-5 years,"5 (Dec 11, 2019)","Zolo Realty, Brokerage",70.00 x 115 Feet,C4651523,5000 sqft,3-Storey,"$10,920 /yr",Detached,20,28 Caswell Drive,Toronto,Willowdale East,"$3,888,888",NaN,NaN,NaN,NaN,NaN
1,No Data,"20 (Nov 26, 2019)","Zolo Realty, Brokerage",33.00 x 120 Feet,E4642347,No Data,2-Storey,"$3,115 /yr",Detached,77,18 Vanbrugh Avenue,Toronto,Birchcliffe-cliffside,"$799,000",NaN,NaN,NaN,NaN,NaN
2,0-5 years,"27 (Nov 19, 2019)","Zolo Realty, Brokerage",32.00 x 107 Feet,E4638092,1500-2000 sqft,2-Storey,"$4,038 /yr",Detached,62,53 Northbrook Road,Toronto,East York,"$1,699,000",NaN,NaN,NaN,NaN,NaN
3,No Data,"1 (Dec 16, 2019)","Right At Home Realty Inc., Brokerage",57.50 x 143.93 Feet,W4654216,No Data,Bungalow,"$3,936 /yr",Detached,68,1023 Kipling Avenue,Toronto,Islington-city Centre West,"$1,550,000",NaN,NaN,NaN,NaN,NaN
4,No Data,"1 (Dec 16, 2019)","One Percent Realty Ltd., Brokerage",55.96 x 189.2 Feet,E4654229,No Data,2-Storey,"$5,215 /yr",Detached,45,82 Whistling Hills Drive,Toronto,Milliken,"$1,198,000",NaN,NaN,NaN,NaN,NaN
